In [1]:
MAX_LEN = 512
TWITTER_ROBERTA_MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
FOLDER_PATH = "/content/drive/MyDrive/Colab Notebooks/Machine learning/ML_project/"
# reduce lr on plateau

In [2]:
import numpy as np
import pandas as pd
import random
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torch.nn import CrossEntropyLoss

import transformers
from transformers import AutoTokenizer, AutoConfig
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import re


d:\APPS\Anaconda\envs\ML\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class ROBERTAModel(nn.Module):
    """
    RoBERTa model with a dropout and linear layer for binary text classification
    """
    def __init__(self, roberta_model, num_classes=2, dropout_rate=0.3):
        super(ROBERTAModel, self).__init__()
        self.roberta = AutoModelForSequenceClassification.from_pretrained(roberta_model)
        self.drop = nn.Dropout(dropout_rate)
        self.out = nn.Linear(self.roberta.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.roberta(input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        output = self.drop(logits)
        return self.out(output)

In [4]:
def preprocess(text):
  # Remove '<user>'
  text = re.sub(r'<user>', '', text)
  # Remove '<url>'
  text = re.sub(r'<url>', '', text)
  # remove numbers
  text = re.sub(r'\d+', '', text)
  return text

In [5]:
def train(data, model, optimizer, device):
    """
        Train the model for one epoch
    """
    model.train()
    running_loss = 0.0

    for batch_idx, d in enumerate(data):
        mask = d['mask'].to(device, dtype=torch.long)
        ids = d['ids'].to(device, dtype=torch.long)
        token_type_ids = d['token_type_ids'].to(device, dtype=torch.long)
        targets = d['targets'].to(device, dtype=torch.long)
        optimizer.zero_grad()
        outputs = model(ids, mask, token_type_ids)

        loss = torch.nn.CrossEntropyLoss(outputs, targets) # Calculate loss
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0 and batch_idx !=0:
            temp = f'Batch index = {batch_idx}\tRunning Loss = {running_loss/10}'
            print(temp)
            running_loss = 0.0

In [7]:
pos_path = 'data/twitter-datasets/train_pos_full.txt'
neg_path = 'data/twitter-datasets/train_neg_full.txt'

with open(pos_path, 'r') as f:
    pos_tweets = f.readlines()
with open(neg_path, 'r') as f:
    neg_tweets = f.readlines()

# TODO: preprocess data
pos_tweets = [preprocess(tweet) for tweet in pos_tweets]
neg_tweets = [preprocess(tweet) for tweet in neg_tweets]


pos_labels = [1 for _ in range(len(pos_tweets))]
neg_labels = [0 for _ in range(len(neg_tweets))]
labels = pos_labels + neg_labels
tweets = pos_tweets + neg_tweets

# Combine labels and tweets into a list of tuples
data = list(zip(tweets, labels))

# Shuffle the data
random.shuffle(data)

# Unpack the shuffled data back into separate lists
train_tweets, train_labels = zip(*data)

# Use RoBERTa tokenizer
tokenizer = AutoTokenizer.from_pretrained(TWITTER_ROBERTA_MODEL)

# Tokenize and convert to input IDs
train_encodings = tokenizer(train_tweets, truncation=True, padding=True, max_length=MAX_LEN, return_tensors="pt")

config.json: 100%|██████████| 929/929 [00:00<?, ?B/s] 
d:\APPS\Anaconda\envs\ML\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ke\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
vocab.json: 100%|██████████| 899k/899k [00:00<00:00, 3.15MB/s]
merges.txt: 100%|██████████| 456k/

In [26]:
class TweetDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)

In [ ]:
# Set k value
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

num_epoch = 1

model = AutoModelForSequenceClassification.from_pretrained(TWITTER_ROBERTA_MODEL)
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3  # Number of training epochs


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
criterion = CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(train_dataset) * (k_folds-1))


# Function to calculate accuracy
def calc_accuracy(preds, labels):
    _, predictions = torch.max(preds, dim=1)
    correct = (predictions == labels).float()
    acc = correct.sum() / len(correct)
    return acc

all_labels = []
all_predictions = []

for fold, (train_indices, val_indices) in enumerate(kf.split(train_dataset)):
    print(f"Fold {fold + 1}/{k_folds}")

    # Create data loaders for the current fold
    train_fold_subset = torch.utils.data.Subset(train_dataset, train_indices)
    val_fold_subset = torch.utils.data.Subset(train_dataset, val_indices)
    train_loader = DataLoader(train_fold_subset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_fold_subset, batch_size=32, shuffle=False)

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        model.train()
        total_train_loss = 0
        total_train_acc = 0

        for step, batch in enumerate(train_loader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = criterion(outputs.logits, batch['labels'])
            acc = calc_accuracy(outputs.logits, batch['labels'])
            print(f"Step {step+1}/{len(train_loader)} - Loss: {loss.item()}, Accuracy: {acc.item()}")
            total_train_loss += loss.item()
            total_train_acc += acc.item()

            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_train_acc = total_train_acc / len(train_loader)
        print(f"Epoch {epoch+1} - Train loss: {avg_train_loss}, Accuracy: {avg_train_acc}")

        # Validation Loop
        model.eval()
        total_val_accuracy = 0
        total_val_loss = 0

        for batch in val_loader:
            with torch.no_grad():
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs = model(**batch)
                loss = criterion(outputs.logits, batch['labels'])
                acc = calc_accuracy(outputs.logits, batch['labels'])

                total_val_loss += loss.item()
                total_val_accuracy += acc.item()

                # for overall metrics calculation
                preds = outputs.logits.argmax(dim=1).cpu().numpy()
                labels = batch['labels'].cpu().numpy()
                all_predictions.extend(preds)
                all_labels.extend(labels)

        avg_val_loss = total_val_loss / len(val_loader)
        avg_val_accuracy = total_val_accuracy / len(val_loader)
        print(f"Epoch {epoch+1} - Validation loss: {avg_val_loss}, Accuracy: {avg_val_accuracy}")

# After all folds, calculate overall metrics
precision = precision_score(all_labels, all_predictions, average='macro')
recall = recall_score(all_labels, all_predictions, average='macro')
f1 = f1_score(all_labels, all_predictions, average='macro')

print("Training complete!")
print(f"Overall Precision: {precision}, Recall: {recall}, F1 Score: {f1}")

### Possible improvement
1. Don't store all the scores in one variable, just store for each epoch and only keep the average score
2. save model for Each epoch

In [28]:
torch.save(model, FOLDER_PATH + 'manipulated/bert.pth')

In [ ]:
## predict
model = torch.load(FOLDER_PATH + 'manipulated/bert.pth')

## use the test set
test_path = FOLDER_PATH +'data/twitter-datasets/test_data.txt'
with open(test_path, 'r') as f:
    test_tweets = f.readlines()

test_encodings = tokenizer(test_tweets, truncation=True, padding=True, max_length=MAX_LEN)
test_dataset = TweetDataset(test_encodings, [0 for _ in range(len(test_tweets))])
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


model.eval()
predictions = []
for batch in test_loader:
    with torch.no_grad():
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = outputs.logits.argmax(dim=1).cpu().numpy()
        predictions.extend(preds)

In [ ]:
predictions = np.array(predictions)
predictions[predictions == 0] = -1

In [ ]:
print(predictions)

In [ ]:
submission = pd.DataFrame({'Id':range(1, len(predictions) + 1),'Prediction': predictions})
submission.to_csv(FOLDER_PATH + 'manipulated/roberta_submission.csv', index=False)